In [30]:
import sys
sys.path.append('/usr/local/bin/python2.7')


import numpy as np
import os, sys, getopt, pickle, csv, sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, make_scorer, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from textblob import TextBlob
import random
import matplotlib.pyplot as plt
from sklearn import metrics
from collections import Counter
import argparse
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score    
import preprocessor as p

In [31]:
models = [ 'svm', 'naive', 'lr', 'random_forest']
NO_OF_FOLDS = 10
MODEL_TYPE = "all"
HASH_REMOVE = None
LABEL_ENCODING_TWITTER = {'racism':0,'sexism':1,'none':2}
LABEL_ENCODING_FORMSPRING = {'none':0,'bully':1}
LABEL_ENCODING_WIKI = {'none':0,'attack':1}

In [45]:
def load_data(filename):
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = []
    for i in range(len(data)):
        if(HASH_REMOVE):
            x_text.append(p.tokenize((data[i]['text']).encode('utf-8')))
        else:
            x_text.append(data[i]['text'])
        labels.append(data[i]['label'])
    return x_text,labels

def get_filename(dataset):
    global N_CLASS, HASH_REMOVE, LABEL_ENCODING
    if(dataset=="twitter"):
        filename = "data/twitter_data.pkl"
        N_CLASS = 3
        LABEL_ENCODING = LABEL_ENCODING_TWITTER
        HASH_REMOVE = False
    elif(dataset=="formspring"):
        N_CLASS = 2
        LABEL_ENCODING = LABEL_ENCODING_FORMSPRING
        filename = "data/formspring_data.pkl"
        HASH_REMOVE = False
    elif(dataset=="wiki"):
        N_CLASS = 2
        LABEL_ENCODING = LABEL_ENCODING_WIKI
        filename = "data/wiki_data.pkl"
        HASH_REMOVE = False
    return filename

In [49]:
def get_scores(y_true, y_pred):
    scores = {}
    for i in range(N_CLASS):
        scores[i] = [
            precision_score(y_true, y_pred, labels=[i], average="micro"), 
            recall_score(y_true, y_pred,  labels=[i], average="micro"),
            f1_score(y_true, y_pred, labels=[i], average="micro")
        ]
    return scores
    
def print_scores(scores):
    for i in range(N_CLASS):
        scores_class_prec = []
        scores_class_rec = []
        scores_class_f1 = []
        for foldscore in scores:
            scores_class_prec.append(foldscore[i])
            scores_class_rec.append(foldscore[i])
            scores_class_f1.append(foldscore[i])
        scores_class_prec = np.array(scores_class_prec)
        scores_class_rec = np.array(scores_class_rec)
        scores_class_f1 = np.array(scores_class_f1)
        class_string = ""
        for key, value in LABEL_ENCODING.items():
            if(value == i):
                class_string = key
        print "Precision Class %s (avg): %0.3f (+/- %0.3f)" % (class_string,scores_class_prec.mean(), scores_class_prec.std() * 2)
        print "Recall Class %s (avg): %0.3f (+/- %0.3f)" % (class_string,scores_class_rec.mean(), scores_class_rec.std() * 2)
        print "F1_score Class %s (avg): %0.3f (+/- %0.3f)" % (class_string,scores_class_f1.mean(), scores_class_f1.std() * 2)


In [28]:
arr = []
arr.append(np.hstack(np.array(["precision","recall","F1"])))
arr.append(np.hstack(np.array(["precision","recall","F1"])))
arr.append(np.hstack(np.array(["precision","recall","F1"])))
arr = np.array(arr)
print "%s" % str(arr[:, 1])

['recall' 'recall' 'recall']


In [47]:
def classification_model(X, Y, model_type):
    X, Y = shuffle(X, Y, random_state=42)
    print "Model Type:", model_type
    kf = KFold(n_splits=NO_OF_FOLDS)
    scores = []
    for train_index, test_index in kf.split(X):
        Y = np.asarray(Y)
        model = get_model(model_type)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        curr_scores = get_scores(y_test, y_pred) 
        #[
        #   [precision, precision, precision], -> length == number of classes
        #   [recall, recall, recall], 
        #   [f1, f1, f1]
        #]
        scores.append(curr_scores)
        # [
        #   [precision, ... , precision] -> len == number of folds
        #   [recall, ... , recall]
        #   [f1, ... , f1]
        # ]
    print_scores(np.array(scores))

In [35]:
def get_model(m_type):
    if m_type == 'lr':
        logreg = LogisticRegression(class_weight="balanced")
    elif m_type == 'naive':
        logreg =  MultinomialNB()
    elif m_type == "random_forest":
        logreg = RandomForestClassifier(n_estimators=100, n_jobs=-1)
    elif m_type == "svm":
        logreg = LinearSVC(class_weight="balanced")
    else:
        print "ERROR: Please specify a correst model"
        return None
    return logreg

In [36]:
def train(x_text, labels, MODEL_TYPE):
    
    if(WORD):
        print("Using word based features")
        bow_transformer = CountVectorizer(analyzer="word",max_features = 10000,stop_words='english').fit(x_text)
        comments_bow = bow_transformer.transform(x_text)
        tfidf_transformer = TfidfTransformer(norm = 'l2').fit(comments_bow)
        comments_tfidf = tfidf_transformer.transform(comments_bow)
        features = comments_tfidf
    else: 
        print("Using char n-grams based features")
        bow_transformer = CountVectorizer(max_features = 10000, ngram_range = (1,2)).fit(x_text)
        comments_bow = bow_transformer.transform(x_text)
        tfidf_transformer = TfidfTransformer(norm = 'l2').fit(comments_bow)
        comments_tfidf = tfidf_transformer.transform(comments_bow)
        features = comments_tfidf
    
    if(data == "twitter"):
        labels = np.array([LABEL_ENCODING_TWITTER[b] for b in labels])
    
    from collections import Counter
    print(Counter(labels))
    
    if(MODEL_TYPE != "all"):
        classification_model(features, labels, MODEL_TYPE)
    else:
        for model_type in models:
            classification_model(features, labels, model_type)

In [ ]:
data = "formspring"
WORD =  False
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using char n-grams based features
Counter({0: 11997, 1: 776})
Model Type: svm
Precision Class 1 (avg): 0.466 (+/- 0.109)
Recall Class 1 (avg): 0.503 (+/- 0.122)
F1_score Class 1 (avg): 0.483 (+/- 0.104)
Model Type: naive
Precision Class 1 (avg): 0.850 (+/- 0.640)
Recall Class 1 (avg): 0.015 (+/- 0.015)
F1_score Class 1 (avg): 0.030 (+/- 0.028)
Model Type: lr
Precision Class 1 (avg): 0.410 (+/- 0.099)
Recall Class 1 (avg): 0.626 (+/- 0.131)
F1_score Class 1 (avg): 0.495 (+/- 0.104)
Model Type: random_forest
Precision Class 1 (avg): 0.735 (+/- 0.230)
Recall Class 1 (avg): 0.161 (+/- 0.084)
F1_score Class 1 (avg): 0.261 (+/- 0.122)


In [ ]:
data = "formspring"
WORD = True
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using word based features
Counter({0: 11997, 1: 776})
Model Type: svm
Precision Class 1 (avg): 0.415 (+/- 0.089)
Recall Class 1 (avg): 0.525 (+/- 0.132)
F1_score Class 1 (avg): 0.463 (+/- 0.100)
Model Type: naive
Precision Class 1 (avg): 0.575 (+/- 0.950)
Recall Class 1 (avg): 0.013 (+/- 0.029)
F1_score Class 1 (avg): 0.025 (+/- 0.055)
Model Type: lr
Precision Class 1 (avg): 0.407 (+/- 0.079)
Recall Class 1 (avg): 0.617 (+/- 0.127)
F1_score Class 1 (avg): 0.489 (+/- 0.084)
Model Type: random_forest
Precision Class 1 (avg): 0.695 (+/- 0.264)
Recall Class 1 (avg): 0.162 (+/- 0.067)
F1_score Class 1 (avg): 0.261 (+/- 0.098)


In [50]:
data = "twitter"
WORD = False
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using char n-grams based features
Counter({2: 11036, 1: 3117, 0: 1937})
Model Type: svm
Precision Class 0 (avg): 0.756 (+/- 0.068)
Recall Class 0 (avg): 0.756 (+/- 0.068)
F1_score Class 0 (avg): 0.756 (+/- 0.068)
Precision Class 1 (avg): 0.760 (+/- 0.069)
Recall Class 1 (avg): 0.760 (+/- 0.069)
F1_score Class 1 (avg): 0.760 (+/- 0.069)
Precision Class 2 (avg): 0.894 (+/- 0.021)
Recall Class 2 (avg): 0.894 (+/- 0.021)
F1_score Class 2 (avg): 0.894 (+/- 0.021)
Model Type: naive
Precision Class 0 (avg): 0.655 (+/- 0.216)
Recall Class 0 (avg): 0.655 (+/- 0.216)
F1_score Class 0 (avg): 0.655 (+/- 0.216)
Precision Class 1 (avg): 0.657 (+/- 0.387)
Recall Class 1 (avg): 0.657 (+/- 0.387)
F1_score Class 1 (avg): 0.657 (+/- 0.387)
Precision Class 2 (avg): 0.881 (+/- 0.131)
Recall Class 2 (avg): 0.881 (+/- 0.131)
F1_score Class 2 (avg): 0.881 (+/- 0.131)
Model Type: lr
Precision Class 0 (avg): 0.759 (+/- 0.069)
Recall Class 0 (avg): 0.759 (+/- 0.069)
F1_score Class 0 (avg): 0.759 (+/

In [51]:
data = "twitter"
WORD = True
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using word based features
Counter({2: 11036, 1: 3117, 0: 1937})
Model Type: svm
Precision Class 0 (avg): 0.760 (+/- 0.078)
Recall Class 0 (avg): 0.760 (+/- 0.078)
F1_score Class 0 (avg): 0.760 (+/- 0.078)
Precision Class 1 (avg): 0.773 (+/- 0.066)
Recall Class 1 (avg): 0.773 (+/- 0.066)
F1_score Class 1 (avg): 0.773 (+/- 0.066)
Precision Class 2 (avg): 0.897 (+/- 0.019)
Recall Class 2 (avg): 0.897 (+/- 0.019)
F1_score Class 2 (avg): 0.897 (+/- 0.019)
Model Type: naive
Precision Class 0 (avg): 0.654 (+/- 0.232)
Recall Class 0 (avg): 0.654 (+/- 0.232)
F1_score Class 0 (avg): 0.654 (+/- 0.232)
Precision Class 1 (avg): 0.664 (+/- 0.364)
Recall Class 1 (avg): 0.664 (+/- 0.364)
F1_score Class 1 (avg): 0.664 (+/- 0.364)
Precision Class 2 (avg): 0.882 (+/- 0.130)
Recall Class 2 (avg): 0.882 (+/- 0.130)
F1_score Class 2 (avg): 0.882 (+/- 0.130)
Model Type: lr
Precision Class 0 (avg): 0.763 (+/- 0.067)
Recall Class 0 (avg): 0.763 (+/- 0.067)
F1_score Class 0 (avg): 0.763 (+/- 0.067)

In [ ]:
data = "wiki"
WORD = False
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using char n-grams based features
Counter({0: 102274, 1: 13590})
Model Type: svm
Precision Class 1 (avg): 0.591 (+/- 0.025)
Recall Class 1 (avg): 0.823 (+/- 0.019)
F1_score Class 1 (avg): 0.688 (+/- 0.018)
Model Type: naive
Precision Class 1 (avg): 0.839 (+/- 0.010)
Recall Class 1 (avg): 0.554 (+/- 0.028)
F1_score Class 1 (avg): 0.667 (+/- 0.021)
Model Type: lr
Precision Class 1 (avg): 0.602 (+/- 0.024)
Recall Class 1 (avg): 0.845 (+/- 0.022)
F1_score Class 1 (avg): 0.703 (+/- 0.017)
Model Type: random_forest


KeyboardInterrupt: 

In [ ]:
data = "wiki"
WORD = True
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)